# gold.ipynb
Calculates business aggregates from our cleaned datasets

In [1]:
from pyspark.sql import SparkSession
from spark_config import configure_spark_session

In [2]:
# Stop any existing spark sessions, from previous jupyter runs
spark = SparkSession.builder.getOrCreate()
spark.stop()

# Create a new spark session
builder = SparkSession.builder
builder.appName("silver")
builder.master("spark://spark-master:7077")

# Apply common spark configs, for the nessie catalog and access to minio
configure_spark_session(builder)

spark = builder.getOrCreate()

In [5]:
spark.sql("""
    CREATE SCHEMA IF NOT EXISTS example.gold
    LOCATION 's3a://gold/'
""")

DataFrame[]

In [7]:
spark.sql("""
CREATE TABLE IF NOT EXISTS example.gold.page_loads_per_day (
    date TIMESTAMP,
    page STRING,
    page_load_count INT
)
USING ICEBERG
""")

25/04/03 00:51:56 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


DataFrame[]

In [10]:
spark.sql(f"""
INSERT INTO example.gold.page_loads_per_day
SELECT
  date,
  page,
  count(*) as page_count
FROM (
  SELECT
    DATE_TRUNC('DAY', event_ts) as date,
    page
  FROM example.silver.page_load
) subquery
GROUP BY date, page
""")

print('Done')


Done


In [11]:
spark.stop()